In [1]:
import pandas as pd
import folium
from IPython.display import display

# Step 1: Load the data
data = pd.read_csv('grid_capacity.csv')

# Placeholder coordinates for cities (replace with actual geocoding if available)
city_coords = {
    "Łomża": (53.1781, 22.0597),
    "Wysokie Mazowieckie": (52.9167, 22.5167),
    "Małkinia": (52.6903, 22.0389),
    "Otwock": (52.1059, 21.2616),
    "Babice": (52.2363, 20.8592),
    "Ołtarzew": (52.2136, 20.7978),
    "Karczew": (52.0833, 21.2333),
    "Mszczonów": (51.9667, 20.5167),
    "Poddębice": (51.8906, 18.9553),
    "Piotrków": (51.4056, 19.7033),
    "Kielce Piaski": (50.8661, 20.6286),
    "Bogdanka": (51.2667, 23.0167),
    "Biłgoraj": (50.5406, 22.7197),
    "Dęblin": (51.5675, 21.8508),
    "Łuków": (51.9292, 22.3792),
    "Puławy": (51.4167, 21.9667),
    "Chełm": (51.1431, 23.4717),
    "Zamość": (50.7239, 23.2511),
    "Stalowa Wola": (50.5744, 22.0535),
    "Opatów": (50.8000, 21.4306)
}

# Map the city names to coordinates
def get_coordinates(city_list):
    for city in city_list.split(', '):
        if city in city_coords:
            return city_coords[city]
    return (0, 0)  # Default if no match found

# Apply the updated function to get coordinates for the first matching city
data['Coordinates'] = data['Cities'].apply(get_coordinates)

# Step 2: Generate the table HTML
table_html = """
<div id="table-overlay" style="
    position: absolute;
    top: 10px;
    left: 50px;
    z-index: 1000;
    background-color: white;
    opacity: 0.9;
    padding: 0;  /* Removed padding around the table */
    border-radius: 8px;
    box-shadow: 0px 2px 6px rgba(0,0,0,0.2);
    width: 250px;
    font-family: Arial, sans-serif;
    font-size: 8px;
">
    <table style="border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; font-size: 12px;">
        <thead>
            <tr style="background-color: #f2f2f2;">
                <th style="border: 1px solid #ddd; padding: 4px; text-align: left; width: 10%;">#</th>
                <th style="border: 1px solid #ddd; padding: 4px; text-align: left; width: 60%;">Group</th>
                <th style="border: 1px solid #ddd; padding: 4px; text-align: left; width: 30%;">2025 MW</th>
            </tr>
        </thead>
        <tbody>
"""
for index, (_, row) in enumerate(data.iterrows(), start=1):  # Add `enumerate` for row index
    table_html += f"""
            <tr>
                <td style="border: 1px solid #ddd; padding: 3px; text-align: left;">{index}</td>
                <td style="border: 1px solid #ddd; padding: 3px; text-align: left;">{row['Group']}</td>
                <td style="border: 1px solid #ddd; padding: 3px; text-align: left;">{row['2025 (MW)']}</td>
            </tr>
    """
table_html += """
        </tbody>
    </table>
</div>
"""


# Step 3: Initialize the map
m = folium.Map(
    location=[52.0, 19.0], 
    zoom_start=6, 
    tiles="CartoDB Positron"
)

# Add circles for each group
for index, row in enumerate(data.iterrows(), start=1):  # Add `enumerate` for group numbering
    _, row_data = row
    lat, lon = row_data['Coordinates']
    if (lat, lon) != (0, 0):  # Skip groups with no valid coordinates
        # Add a circle marker
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,  # Fixed radius for consistent size
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=f"{row_data['Group']}: {row_data['2025 (MW)']} MW"
        ).add_to(m)

        # Determine position of the number (above for groups 4 and 5, below for others)
        if index in [4, 5]:  # Groups 4 and 5
            text_lat = lat + 0.1  # Position number above the circle
        else:
            text_lat = lat - 0.1  # Position number below the circle

        # Add a label with the group number
        folium.map.Marker(
            [text_lat, lon],  # Adjust latitude based on the condition
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 10px; text-align: center; color: black;">{index}</div>"""
            ),
        ).add_to(m)

# Step 4: Add the table overlay to the map
m.get_root().html.add_child(folium.Element(table_html))

# Step 5: Display the map
display(m)


/Users/lukasvonkoch/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
